In [ ]:
# load dependencies
## geospatial data handling
library(rgdal)
library(raster)
library(sp)
library(mapdata)
library(maptools)
library(ncdf4)
## general data handling
library(XML)
library(dplyr)
library(tidyr)
library(reshape2)
library(downloader)
## plotting
library(directlabels)
library(rasterVis)
library(ggplot2)
## for display reasons
library(knitr)
library(kableExtra)
library(IRdisplay)
library(repr)
options(repr.plot.wdith=6, repr.plot.height=8)

---
During the second part of the tutorial, we focus on the "Estuaires picards et mer d'Opale" Marine Protected Area (MPA) in France. The shapefile for this area can be downloaded from the [World Database on Protected Areas (WDPA)](https://www.protectedplanet.net/555559631).

In [ ]:
# download and unzip the shapefile
con = "http://d1gam3xoknrgr2.cloudfront.net/current/WDPA_WDOECM_protected_area_555559631_shp.zip"
download(con,"shapefile.zip", mode = "wb")
unzip("shapefile.zip",exdir="MPA2")
file.remove("shapefile.zip")
unzip("MPA2/WDPA_WDOECM_protected_area_555559631_shp0.zip", exdir = "MPA2")
# read it into R
mpa = readOGR("MPA2","WDPA_WDOECM_protected_area_555559631_shp-polygons")
# get the spatial extent of the MPA
xmin <- extent(mpa)@xmin
ymin <- extent(mpa)@ymin
xmax <- extent(mpa)@xmax
ymax <- extent(mpa)@ymax

We make some maps to show the location of the MPA.

In [ ]:
# Localisation at global scale
map("worldHires",col="light grey",fill=T)
points(coordinates(mpa),cex=2,col="blue",pch="+")
title(paste("Region of interest ",
            "( W-Lon",round(xmin,2),
            "S-Lat",round(ymin,2),
            "E-Lon",round(xmax,2),
            "N-Lat",round(ymax,2),")"),cex=.5)

# Localisation at local scale
plot(mpa,xlim=c(xmin-1,xmax+1),ylim=c(ymin-1,ymax+1),axes=T,col="red")
map("worldHires",add=T,col="light grey",fill=T)
plot(mpa,add=T,col="blue")
title(paste("MPA",mpa$ORIG_NAME),cex=.5)

We again extract the bathymetry using the function defined in part 1 of this tutorial.

In [ ]:
# Define a function to read in raster data from the EMODnet bathymetry WCS
getbathymetry<-function (name = "emodnet:mean", xmin = 15, xmax = 20.5, ymin = 30, ymax = 32.5){
  bbox <- paste(xmin, ymin, xmax, ymax, sep = ",")
                              
  con <- paste("https://ows.emodnet-bathymetry.eu/wcs?service=wcs&version=1.0.0&request=getcoverage&coverage=",
               name,"&crs=EPSG:4326&BBOX=", bbox,
               "&format=image/tiff&interpolation=nearest&resx=0.00208333&resy=0.00208333", sep = "")
  
  print(con)
  
  stop   
  nomfich <- paste(name, "img.tiff", sep = "_")
  nomfich <- tempfile(nomfich)
  download(con, nomfich, quiet = TRUE, mode = "wb")
  img <- raster(nomfich)
  img[img == 0] <- NA
  img[img < 0] <- 0
  names(img) <- paste(name)
  return(img)
}

# get the bathymetry data for the MPA
bathy_img <- getbathymetry(name = "emodnet:mean", xmin, xmax, ymin, ymax)
bathy <- as.data.frame(as(bathy_img, "SpatialPixelsDataFrame"))

# Working with data from EMODnet biology 

## Access polygon data with WFS

Using the EMODnet biology WFS we can get the zooplankton abundances for different grid cells within the MPA (bbox). To calculate statistics of the abundance data over a particular period and a particular season, we can download it as a geoJSON file.

In [ ]:
# set the request parameters
reg.name = 36317
spec.name = 'Acartia_spp'
trend_start = 2004
trend_end = 2013
season.code = 1
bbox <- paste(xmin, ymin, xmax, ymax, sep = ",")
type.name='Emodnetbio:OOPS_products_vliz'

# build the WFS URL
url<-"http://geo.vliz.be/geoserver/Emodnetbio/ows?service=WFS&version=1.0.0&request=GetFeature&outputFormat=json"
full_url<-paste(url,"&viewParams=;startYearCollection:",trend_start,";mrgid:",reg.name,";scientificName:",spec.name,";season:",season.code,"&typeName=",type.name,"&BBOX=",bbox,sep="")

# get the data as a geoJSON file
library("rjson")
json_data <- fromJSON(file=full_url)
# to display the file
# str(json_data)
# to save
# geojson_write(json_data, file = 'MyJson.geojson') #bogue ?

test<-json_data[['features']]
mean=0 
l<- length(test)
for (i in 1:l){
tests<-test[[l]]
mean<-mean+tests$properties$measurementValue
}
mean_value_season1<-mean/l
mean_value_season1

We can also visualise the extent of the zooplankton abundance grid cell by downloading the data as a zipped shapefile and plotting it on a map.


In [ ]:
# build the WFS URL
url<-"http://geo.vliz.be/geoserver/Emodnetbio/ows?service=WFS&version=1.0.0&request=GetFeature&outputFormat=shape-zip"
full_url<-paste(url,"&viewParams=;startYearCollection:",trend_start,";mrgid:",reg.name,";scientificName:",spec.name,";season:",season.code,"&typeName=",type.name,"&BBOX=",bbox,sep="")

# get the data as a zipped shapefile
download(full_url, dest="shapefile.zip", mode="wb") 
unzip("shapefile.zip", overwrite = TRUE)
OOPS_products <- readOGR(dsn = ".", layer = "OOPS_products_vlizPolygon")

# plot on a map
map<-ggplot()+
  theme_bw()+
  theme(panel.grid.minor.y= element_blank(), panel.grid.minor.x = element_blank())+
  geom_raster(data=bathy,aes(x=x,y=y,fill=emodnet.mean),alpha=.75)+
  scale_fill_gradient(low = "white", high = "darkblue",name="Depth (m)")+
  geom_polygon(data=OOPS_products,aes(x=long,y=lat,group=group, fill="OOPS_products"),colour="green",fill="blue",alpha=.1)+
  coord_quickmap(xlim=range(xmin,xmax),ylim=range(ymin,ymax))+
ggtitle("OOPS_products polygons")+xlab("Longitude")+ylab("Latitude")
plot(map)

# export to a shapefile which can then be visualised in e.g. QGIS
writeOGR(OOPS_products,"OOPS_products","OOPS_products_bbox",driver="ESRI Shapefile",overwrite_layer = TRUE)

Alternatively, we can also download the zooplankton abundance grid cell as geoJSON.

In [ ]:
# function to read GeoJSON
getgeojson<-function(name="Emodnetbio:OOPS_products_vliz",xmin=-1,xmax=1,ymin=49,ymax=50){
  reg.name = 36317 #label="Greater North Sea"
  trend_start = 1958 #startDate
  trend_end = 2013 #endDate
  spec.name = 'Acartia_spp'
  trend_start = 2004
  trend_end = 2013
  season.code = 1
  bbox<-paste(xmin,ymin,xmax,ymax,sep=",")
  
  url<-"http://geo.vliz.be/geoserver/Emodnetbio/ows?service=WFS&version=1.0.0&request=GetFeature&outputFormat=json"
  con<-paste(url,"&viewParams=;startYearCollection:",trend_start,";mrgid:",reg.name,";scientificName:",spec.name,";season:",season.code,"&typeName=",name,"&BBOX=",bbox,sep="")
  print(con)
  temp <- tempfile()
  download(con,temp)
  ogrInfo(dsn=temp)
  layer<-readOGR(dsn=temp)
  return(layer)
}
OOPS_products<-getgeojson(name="Emodnetbio:OOPS_products_vliz",xmin,xmax,ymin,ymax)

map <- ggplot()+
  theme_bw()+
  theme(panel.grid.minor.y= element_blank(), panel.grid.minor.x = element_blank()) +
  geom_raster(data=bathy,aes(x=x,y=y,fill=emodnet.mean),alpha=.75) +
  scale_fill_gradient(low = "white", high = "darkblue",name="Depth (m)") +
  geom_polygon(data=OOPS_products,aes(x=long,y=lat,group=group,fill="OOPS_products"),colour="green",fill="blue",alpha=.1) +
  coord_quickmap(xlim=range(xmin,xmax),ylim=range(ymin,ymax)) +
  ggtitle("OOPS_products polygons")+xlab("Longitude")+ylab("Latitude")

plot(map)

## Access time series data with WFS

Using WFS, we can also access the zooplankton abundance time series as a csv file.


In [ ]:
# determine parameters
reg.name = 36317 #label="Greater North Sea"
spec.name = 'Acartia_spp' #species
trend_start = 1958 #startDate
trend_end = 2013 #endDate

# construct WFS url
mainurl<-"http://geo.vliz.be/geoserver/Emodnetbio/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=Emodnetbio:OOPS_summaries&outputFormat=csv&SQL_FILTER=startYearCollection-endYearCollection=0"
fullurl<-paste(mainurl,"ANDscientificName='",spec.name,"'ANDmrgid=",reg.name,sep="")

# get the data
datans <- read.csv(file= fullurl, header=TRUE)

We now analyse the time series seasonality and trends using the `stl()` loess seasonal decomposition function.

In [ ]:
levels(datans$scientificName)[levels(datans$scientificName)=="RatioLS copepods"] <- "Large/Small copepod ratio"
seldata<-datans[with(datans, order(datans$startYearCollection,datans$season)),]
# convert to a time series object
tscf<-ts(data=seldata$avg_ab, start=trend_start, end=trend_end, frequency=4)
# decompose the time series
fitstl<-stl(tscf,s.window="periodic",t.window=15)
plot(fitstl)

# Search for data using EMODnet Catalogue Service

Using the EMODnet Catalogue Service for the Web (CSW), we can query and search collections of metadata for data, services and information objects related to the EMODnet Marine Data. In the example below, we use the `XML` package to send a request to the CSW endpoint, parse the XML response and store it in a dataframe.

In [ ]:
# CSW request
con = "https://www.emodnet.eu/geonetwork/emodnet/eng/csw?REQUEST=GetRecords&SERVICE=CSW&VERSION=2.0.2&ELEMENTSETNAME=summary&OUTPUTSCHEMA=http://www.opengis.net/cat/csw/2.0.2&CONSTRAINTLANGUAGE=FILTER&CONSTRAINT_LANGUAGE_VERSION=1.1.0&RESULTTYPE=results&TYPENAMES=csw:Record&CONSTRAINT=%3Cogc:Filter%20xmlns:ogc=%22http://www.opengis.net/ogc%22%3E%3Cogc:PropertyIsEqualTo%3E%3Cogc:PropertyName%3Edc:type%3C/ogc:PropertyName%3E%3Cogc:Literal%3Edataset%3C/ogc:Literal%3E%3C/ogc:PropertyIsEqualTo%3E%3C/ogc:Filter%3E&maxRecords=1000"
download(con,"cswrequest.xml", mode = "wb")


# get and parse response to a dataframe
response = xmlParse("cswrequest.xml")
response = xmlToList(response)
results = data.frame()
for(record in response$SearchResults[names(response$SearchResults) == "SummaryRecord"]){
  # cast to datafame, negating empty fields
  df = as.data.frame(Filter(Negate(is.null),record),  stringsAsFactors = FALSE)
  # combine multiple subjects into 1 column
  if(sum(grepl("subject",names(df))) > 1){
    df = unite(df,"subject",contains("subject"), sep = ", ", remove = TRUE)
  }
  # combine multiple formats into 1 column
  if(sum(grepl("format",names(df))) > 1){
    df = unite(df,"format",contains("format"), sep = ", ", remove = TRUE) 
  }
  results = bind_rows(results,df)
}

Now that we have all the records stored in a dataframe, we can perform detailed queries.

In [ ]:
# look for Records with keyword 'Habitat'
results %>%
  filter(grepl("Habitat",results$subject)) %>%
  # limit printing length of abstract
  mutate(abstract = paste(substring(abstract,0,80),"...")) %>%
  knitr::kable(floating.environment="sidewaystable") %>%
  kable_styling(bootstrap_options = c("condensed","striped","hover")) %>%
  as.character() %>% display_html()

In [ ]:
# look for Records with keyword 'substrate'
results %>%
  filter(grepl("substrate",results$subject)) %>%
  mutate(abstract = paste(substring(abstract,0,80),"...")) %>%  # limit printing length of abstract
  knitr::kable(floating.environment="sidewaystable") %>%
  kable_styling(bootstrap_options = c("condensed","striped","hover")) %>%
  as.character() %>% display_html()

### [>> Back to index](./index.ipynb)

-----------------------------------------------------------------------------------------------------------
<center> Provided by EMODnet. See our <a href=https://emodnet.eu/en/terms-use> terms of use </a></center>
<center><a href="https://www.emodnet.eu/"><img style="float: None" style="border-width:0" src="https://www.emodnet.eu/sites/emodnet.eu/files/public/logo_2x_1.png" /></a>
</center>